In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_4월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(821105, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
821105


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0,"[04, 01, 생일, 맞이, 강원도, 여행, 의, 시작, 후후, 제일, 기대하던,...","[('04', 'Number'), ('01', 'Number'), ('생일', 'N..."
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0,"[호텔, 먼저, 들어가서, 체크, 인, 선착순, 티켓, 받아오고, 밥, 먹으러, 고...","[('호텔', 'Noun'), ('먼저', 'Noun'), ('들어가다', 'Ver..."
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0,"[진짜, 바다, 도, 너무, 푸르고, 그저, 행복, 잠깐, 기다리면서, 애, 들, ...","[('진짜', 'Noun'), ('바다', 'Noun'), ('도', 'Josa')..."
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0,"[근데, 진짜, 날씨, 가, 너무, 좋아서, ㅠㅠㅠ, 완벽했던, 여행, 바다, 가,...","[('근데', 'Adverb'), ('진짜', 'Noun'), ('날씨', 'Nou..."
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0,"[그리고, 바닷가, 에서, 벗어나니까, 막, 춥진, 않은, 날씨, 라서, 식당, 앞...","[('그리고', 'Conjunction'), ('바닷가', 'Noun'), ('에서..."
...,...,...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280,"[분석, 글, 은, 저, 의, 투자, 시점, 과, 생각, 이고, 흐름, 만, 파악,...","[('분석', 'Noun'), ('글', 'Noun'), ('은', 'Josa'),..."
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280,"[참고, 만, 하시고, 투자, 손실, 의, 책임, 은, 본인, 에, 있습니다]","[('참고', 'Noun'), ('만', 'Noun'), ('하다', 'Verb')..."
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280,"[같이, 보면, 좋은, 포스팅, 국내, 주식, 사료, 관련, 주, 안녕하세요, 여러...","[('같이', 'Adverb'), ('보다', 'Verb'), ('좋다', 'Adj..."
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280,"[오늘, 정리, 해볼, 테마, 는, 사료, 관련, 주입, 니, 다, 사료, m, b...","[('오늘', 'Noun'), ('정리', 'Noun'), ('해보다', 'Verb..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(147212, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '등', '때', '우리', '말', '더', '문화', '길', '생각', '나', '나무', '내', '위', '사진', '벚꽃', '시간', '공원', '층', '전', '제', '의', '중', '안', '때문', '꽃', '개', '일', '마을', '하나', '아이', '위해', '집', '시작', '공간', '날', '저', '서울', '여행', '로', '한국', '카페', '및', '작품', '다른', '모습']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 821105/821105 [00:05<00:00, 156910.75it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

104
85


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 821105/821105 [10:05<00:00, 1355.15it/s] 


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [11]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/4월/단어동시출현_딕셔너리_4월.csv", encoding='UTF-8')

In [12]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 176167589/176167589 [20:32<00:00, 142941.16it/s] 


(176167589, 3)


,term1,term2,freq
0,시작,해야하나,5
1,사고,시작,88
2,맞이,시작,143
3,시작,에,9393
4,시기,시작,358


In [13]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/4월/단어동시출현_df_4월.csv", index=False, encoding='UTF-8')

In [14]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 176167589/176167589 [36:14<00:00, 81019.86it/s]


,term1,term2,freq
0,내,시작,1220
1,다른,시작,534
2,시작,이,11287
3,시작,의,9642
4,로,시작,4892
...,...,...,...
1220,공원,층,318
1221,공원,다른,324
1222,작품,카페,206
1223,아이,한국,353


In [15]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/4월/단어동시출현_df_상위50_4월.csv", index=False, encoding='UTF-8')

In [16]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1518.53it/s]


In [17]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,등,때,우리,말,더,문화,길,생각,나,나무,내,위,사진,벚꽃,시간,공원,층,전,제,의,중,안,때문,꽃,개,일,마을,하나,아이,위해,집,시작,공간,날,저,서울,여행,로,한국,카페,및,작품,다른,모습
것,0.0,24035.0,22821.0,88432.0,8146.0,13982.0,7306.0,11437.0,11114.0,10811.0,8636.0,4766.0,3695.0,9972.0,11423.0,4780.0,9132.0,6679.0,3465.0,2121.0,4938.0,1745.0,1318.0,5900.0,4594.0,65399.0,5952.0,6786.0,7649.0,2745.0,3068.0,7345.0,1857.0,5053.0,3277.0,4911.0,3130.0,3928.0,2640.0,3313.0,4374.0,1496.0,2158.0,25790.0,2656.0,1379.0,1794.0,2751.0,5419.0,3363.0
수,24035.0,0.0,13220.0,58086.0,10363.0,8747.0,9254.0,7166.0,6927.0,6213.0,6782.0,5754.0,4544.0,5858.0,8237.0,4947.0,6244.0,5150.0,3776.0,2947.0,5384.0,3628.0,2424.0,5056.0,4302.0,49808.0,4887.0,4585.0,5237.0,3151.0,3367.0,5152.0,2450.0,3201.0,3751.0,3366.0,2623.0,2691.0,4966.0,2411.0,2911.0,2063.0,2500.0,23371.0,2315.0,2057.0,2925.0,2987.0,3756.0,3357.0
그,22821.0,13220.0,0.0,48756.0,7051.0,9867.0,4526.0,7868.0,6380.0,7042.0,4328.0,2922.0,2816.0,5055.0,7106.0,3066.0,5656.0,4614.0,2055.0,992.0,2804.0,1039.0,890.0,4081.0,3016.0,46537.0,5113.0,4673.0,4298.0,1730.0,2002.0,4560.0,1449.0,3172.0,1759.0,3088.0,2367.0,2481.0,1588.0,2461.0,2351.0,977.0,1135.0,18096.0,1633.0,885.0,1100.0,2442.0,3106.0,2619.0
이,88432.0,58086.0,48756.0,0.0,38084.0,35500.0,25578.0,29467.0,21854.0,22388.0,20134.0,15118.0,14276.0,20857.0,26836.0,16603.0,21111.0,16056.0,14191.0,11265.0,15713.0,9574.0,6307.0,17366.0,14347.0,168786.0,16839.0,17819.0,16055.0,12109.0,11651.0,16255.0,8910.0,11509.0,10499.0,11150.0,10609.0,11287.0,10840.0,12826.0,12676.0,6276.0,7327.0,75797.0,8249.0,7057.0,6554.0,8873.0,11505.0,11315.0
곳,8146.0,10363.0,7051.0,38084.0,0.0,4353.0,3736.0,3787.0,2374.0,1890.0,2890.0,3325.0,3799.0,2484.0,3577.0,2853.0,2793.0,2191.0,3106.0,3002.0,2758.0,3347.0,1218.0,2593.0,2390.0,21422.0,3206.0,2490.0,1617.0,2044.0,1907.0,1840.0,2840.0,1701.0,2655.0,1584.0,1816.0,1429.0,2509.0,1571.0,2324.0,1790.0,3319.0,12423.0,1176.0,2198.0,898.0,795.0,2142.0,1788.0
사람,13982.0,8747.0,9867.0,35500.0,4353.0,0.0,2362.0,4609.0,3892.0,4622.0,3210.0,1918.0,2044.0,4179.0,5378.0,2046.0,3757.0,2369.0,2276.0,1374.0,2037.0,959.0,496.0,2183.0,1603.0,23783.0,2411.0,3039.0,2614.0,1242.0,1235.0,3229.0,1291.0,1832.0,999.0,2106.0,1630.0,1415.0,1080.0,1593.0,1907.0,706.0,889.0,10009.0,996.0,773.0,529.0,825.0,3030.0,1636.0
등,7306.0,9254.0,4526.0,25578.0,3736.0,2362.0,0.0,2447.0,1664.0,2037.0,1642.0,4071.0,1643.0,1217.0,3502.0,2613.0,2558.0,3029.0,1268.0,897.0,1622.0,1955.0,1114.0,3119.0,2799.0,25510.0,2773.0,1582.0,1373.0,1402.0,2530.0,2033.0,1430.0,1038.0,1155.0,1936.0,1184.0,1411.0,2060.0,934.0,989.0,1499.0,906.0,11698.0,1939.0,943.0,3764.0,1634.0,1230.0,1155.0
때,11437.0,7166.0,7868.0,29467.0,3787.0,4609.0,2447.0,0.0,3490.0,3511.0,2779.0,1485.0,1640.0,3232.0,4723.0,1827.0,3569.0,1934.0,1802.0,1125.0,1987.0,745.0,528.0,2424.0,2123.0,20165.0,2181.0,2705.0,2073.0,1257.0,1418.0,2613.0,822.0,1673.0,1367.0,1559.0,1631.0,1810.0,818.0,1665.0,1810.0,700.0,1160.0,9894.0,962.0,679.0,582.0,742.0,1678.0,1354.0
우리,11114.0,6927.0,6380.0,21854.0,2374.0,3892.0,1664.0,3490.0,0.0,3022.0,2371.0,2246.0,1409.0,2856.0,3273.0,1536.0,2461.0,1937.0,1247.0,679.0,1746.0,576.0,516.0,1758.0,1223.0,20100.0,1664.0,2518.0,1794.0,934.0,913.0,2347.0,747.0,1505.0,1510.0,1753.0,1872.0,1195.0,768.0,1288.0,1146.0,496.0,793.0,7460.0,963.0,560.0,524.0,650.0,1415.0,1115.0
말,10811.0,6213.0,7042.0,22388.0,1890.0,4622.0,2037.0,3511.0,3022.0,0.0,2016.0,1225.0,1118.0,2577.0,3831.0,1206.0,3333.0,1709.0,733.0,622.0,1105.0,548.0,268.0,1804.0,1440.0,17696.0,1785.0,2199.0,1554.0,865.0,922.0,2070.0,532.0,1302.0,914.0,1186.0,997.0,996.0,467.0,1058.0,1215.0,443.0,474.0,7100.0,718.0,273.0,520.0,549.0,1340.0,771.0


In [18]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/4월/공출현빈도_df_4월.csv", encoding='UTF-8')